# EXAMPLE DATA

In [1]:
underlyingPosition = ['Long', 1]     # 'Long' or 'Short'; quantity
optPositions = [['Call', 60, 1, -1],
                ['Put', 40, 1, 1]]      # call or put; strike price; maturity; quantity

S = 50          # price of underlying
sigma = 0.4     # volatility of underlying
r = 0.02        # risk-free rate
q = 0.03        # dividend yield

# FUNCTIONS

In [2]:
import numpy as np
from scipy.stats import norm

def callBS(S, K, T, sigma, r, q):
    def f(s):
        s = s if s != 0 else 1.0e-6
        d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return np.exp(-q * T) * s * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)

    return (
        np.array([f(s) for s in S])
        if (isinstance(S, list) or isinstance(S, np.ndarray))
        else f(S)
    )

def callDelta(s, K, T, sigma, r, q):
    s = s if s != 0 else 1.0e-6
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return np.exp(-q * T) * norm.cdf(d1)

def putBS(S, K, T, sigma, r, q):
    def f(s):
        s = s if s != 0 else 1.0e-6
        d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return np.exp(-r * T) * K * norm.cdf(-d2) - np.exp(-q * T) * s * norm.cdf(-d1)

    return (
        np.array([f(s) for s in S])
        if (isinstance(S, list) or isinstance(S, np.ndarray))
        else f(S)
    )

def putDelta(s, K, T, sigma, r, q):
    s = s if s != 0 else 1.0e-6
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return -np.exp(-q * T) * norm.cdf(-d1)

class Option(list):
    def add(self, security, quantity, strike=None, time=None):
        self.append(
            dict(security=security, quantity=quantity, strike=strike, time=time)
        )

    def strikes(self):
        return [d["strike"] for d in self if d["security"] != "Underlying"]

    def grid(self, S):
        strikes = self.strikes()
        maxgrid = 2 * S if len(strikes) == 0 else np.max((2 * S, 1.5 * np.max(strikes)))
        return np.arange(0.5, maxgrid+0.5, 0.5)

    def calculate(self, S, sigma, r, q):
        grid = self.grid(S)
        value = 0
        value_line = np.zeros(len(grid))
        delta = 0
        for x in self:
            if x["security"] == "Underlying":
                value += x["quantity"] * S
                value_line += x["quantity"] * grid
                delta += x["quantity"]
            elif x["security"] == "Call":
                K = x["strike"]
                T = x["time"]
                value += x["quantity"] * callBS(S, K, T, sigma, r, q)
                value_line += x["quantity"] * callBS(grid, K, T, sigma, r, q)
                delta += x["quantity"] * callDelta(S, K, T, sigma, r, q)
            else:
                K = x["strike"]
                T = x["time"]
                value += x["quantity"] * putBS(S, K, T, sigma, r, q)
                value_line += x["quantity"] * putBS(grid, K, T, sigma, r, q)
                delta += x["quantity"] * putDelta(S, K, T, sigma, r, q)
        delta_line = delta * grid + value - delta * S
        return value, value_line, delta, delta_line

# FIGURE

In [3]:
import plotly.graph_objects as go

# Initialize class
x = Option()

if underlyingPosition[0] == "Long":
    x.add(security="Underlying", quantity=underlyingPosition[1])
elif underlyingPosition[0] == "Short":
    x.add(security="Underlying", quantity=-underlyingPosition[1])

for opt in optPositions:
    x.add(security=opt[0], strike=opt[1], time=opt[2], quantity=opt[3])

# Calculate delta and cash position
grid = x.grid(S)
value, value_line, delta, delta_line = x.calculate(S, sigma, r, q)
cash = value - delta*S   

# Figure
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=grid, y=delta_line, mode="lines", name="Delta Hedge", line=dict(color='red')
    )
)
fig.add_trace(
    go.Scatter(
        x=grid, y=value_line, mode="lines", name="Portfolio", line=dict(color='blue')
    )
)
fig.update_layout(hovermode="x unified")
fig.update_yaxes(title=None)
fig.update_layout(yaxis_tickformat=",.2f", xaxis_tickformat=",.2f")
fig.update_xaxes(title="Underlying Price")
fig.show()

print(f'For underlying price of ${S:.0f}, portfolio delta is {delta:.3f} and a cash position in delta hedge of {cash:.2f}')
 

For underlying price of $50, portfolio delta is 0.397 and a cash position in delta hedge of 29.08
